In [24]:
import pandas as pd
emo = pd.read_csv('emo.csv', encoding='latin_1')
emo.dropna()
emo.head()

,Anonymized Message,Valence,Arousal
0,"Remember what she said in my last letter? """,3.00,3.00
1,If I wasn't working here.,2.80,3.10
2,"..""",3.00,3.00
3,Goodwill helps people get off of public assist...,3.44,3.00
4,Sherry learned through our Future Works class ...,3.55,3.27


In [25]:
fb = pd.read_csv('fbemo.csv',  names = ['Score', 'Review'], sep=',', encoding='latin_1')
fb.dropna()
fb.head()

,Anonymized Message,Valence1,Valence2,Arousal1,Arousal2
0,We'll be off and running to a lil' place calle...,9.0,9.0,8.0,8.0
1,I really wana move soon!,4.0,5.0,5.0,7.0
2,thinking at charity cup,5.0,5.0,1.0,1.0
3,"thinks that for girls, boys are easily found. ...",4.0,3.0,6.0,7.0
4,Our Wedding Anniversary tonight...,7.0,7.0,4.0,5.0


In [26]:
fb_list = [fb.columns.values.tolist()] + fb.values.tolist()
emo_list = [emo.columns.values.tolist()] + emo.values.tolist()

In [27]:
import numpy as np
valence_1 = np.zeros(len(emo_list)-1)
arousal_1 = np.zeros(len(emo_list)-1)
i = 1
while i < int(len(emo_list)):
    if emo_list[i][1] >= 3:
        valence_1[i-1] = 1 #for positive valence
    else:
        valence_1[i-1] = 0 #for negative valence
    if emo_list[i][2] >= 3:
        arousal_1[i-1] = 1 #for positive arousal
    else:
        arousal_1[i-1] = 0 #for negative arousal
    i+=1
valence_2 = np.zeros(len(fb_list)-1)
arousal_2 = np.zeros(len(fb_list)-1)
i = 1
while i < int(len(fb_list)):
    if (fb_list[i][1] + fb_list[i][2])/2 >= 5:
        valence_2[i-1] = 1 #for positive valence
    else:
        valence_2[i-1] = 0 #for negative valence
    if (fb_list[i][3] + fb_list[i][4])/2 >= 5:
        arousal_2[i-1] = 1 #for positive arousal
    else:
        arousal_2[i-1] = 0 #for negative arousal
    i+=1
print(valence_1.shape)
print(valence_2.shape)

(10091,)
(2895,)


In [28]:
emo_sentence = [emo_list[x+1][0] for x in range(len(emo_list)-1)]
fb_sentence = [fb_list[x+1][0] for x in range(len(fb_list)-1)]
sentence = emo_sentence + fb_sentence

In [39]:
valence_np = np.hstack((valence_1, valence_2))
arousal_np = np.hstack((arousal_1, arousal_2))
valence = valence_np.tolist()
arousal = arousal_np.tolist()

from sklearn.model_selection import train_test_split
S_train, S_test, v_train, v_test = train_test_split(sentence, valence, test_size=0.3, random_state = 0) 
a_train, a_test = train_test_split(arousal, test_size=0.3, random_state = 0) 
print(len(S_train))
print(len(v_train))
print(len(S_test))
print(len(a_test))

9090
9090
3896
3896


In [40]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
from nltk.corpus import stopwords
def tokenize(sentence):
    tokens = tokenizer.tokenize(sentence)
    toReturn = [stemmer.stem(item.lower()) for item in tokens]
    return toReturn

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
count_vect = TfidfVectorizer(ngram_range=(1,3), tokenizer=tokenize)
trainX = count_vect.fit_transform(S_train)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(a_train)
target_labels = le.classes_
trainy = le.transform(a_train)

In [17]:
def train_random_forest(X,y):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    random = RandomForestClassifier(random_state=0)
    n_list = [500]
    parameters_random = {'n_estimators' : n_list}
    grid = GridSearchCV(random, parameters_random, cv=3)
    grid.fit(X, y)
    print("Best cross-validation score: {:.2f}".format(grid.best_score_))
    print("Best parameters: ", grid.best_params_)
    print("Best estimator: ", grid.best_estimator_)
    cls_random = grid.best_estimator_
    cls_random.fit(X, y)
    return cls_random

def train_classifier(X, y):
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import GridSearchCV
    param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
    print("grid search start")
    grid = GridSearchCV(LogisticRegression(random_state=0, solver='lbfgs',class_weight = 'balanced', max_iter=10000), param_grid, cv=5)
    print("done grid search")
    grid.fit(X, y)
    print("Best cross-validation score: {:.2f}".format(grid.best_score_))
    print("Best parameters: ", grid.best_params_)
    print("Best estimator: ", grid.best_estimator_)
    cls = grid.best_estimator_
    cls.fit(X, y)
    return cls

In [18]:
cls_random = train_random_forest(trainX, trainy)

Best cross-validation score: 0.64
Best parameters:  {'n_estimators': 500}
Best estimator:  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)


In [19]:
cls = train_classifier(trainX, trainy)

grid search start
done grid search
Best cross-validation score: 0.65
Best parameters:  {'C': 10}
Best estimator:  LogisticRegression(C=10, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=10000,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=0,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)


In [27]:
test_list = ['I learned a lot from this class.']
test_list = count_vect.transform(test_list)
yp = cls.predict(test_list)
yq = cls_random.predict(test_list)
print(yp)
print(yq)

[0]
[0]


In [ ]:
def evaluate(X, yt, cls, name='data'):
    """Evaluated a classifier on the given labeled data using accuracy."""
    from sklearn import metrics
    yp = cls.predict(X)
    acc = metrics.accuracy_score(yt, yp)
    print("  Accuracy on %s  is: %s" % (name, acc))
    return acc

In [ ]:
count_vect = TfidfVectorizer(ngram_range=(1,3), tokenizer=tokenize)
testX = count_vect.fit_transform(S_test)
le = preprocessing.LabelEncoder()
le.fit(v_test)
target_labels = le.classes_
testy = le.transform(a_test)
print("\nEvaluating")
evaluate(trainX, trainy, cls, 'train')
acc = evaluate(testX, testy, cls, 'dev')